<a href="https://colab.research.google.com/github/XingxinHE/Python-for-Data-Analysis-Notes/blob/master/8_1_Hierarchical_Indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis.

就是index有两层+

In [2]:
import pandas as pd
import numpy as np


In [3]:
data = pd.Series(np.random.randn(9),
    index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], 
    [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -1.097269
   2    2.204310
   3   -0.079927
b  1   -0.693085
   3    1.830876
c  1    0.473403
   2   -0.738803
d  2    1.001193
   3    1.765319
dtype: float64

In [4]:
#观察上面，其实第一层的index是重复的（比方说aaa,对应123），这样更好理解
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
data['b' : 'c']#第一级index 会带上所有第二级的index

b  1   -0.693085
   3    1.830876
c  1    0.473403
   2   -0.738803
dtype: float64

In [7]:
data.loc[:, 2] #因为这个是series，所以第二个2并不是column，而是第二级index的索引

a    2.204310
c   -0.738803
d    1.001193
dtype: float64

In [8]:
data.unstack() #解开多重索引

,1,2,3
a,-1.097269,2.204310,-0.079927
b,-0.693085,NaN,1.830876
c,0.473403,-0.738803,NaN
d,NaN,1.001193,1.765319


In [9]:
data.unstack().stack() #对应的是stack

a  1   -1.097269
   2    2.204310
   3   -0.079927
b  1   -0.693085
   3    1.830876
c  1    0.473403
   2   -0.738803
d  2    1.001193
   3    1.765319
dtype: float64

In [10]:
#DataFrame也可以有多重index
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
    columns=[['Ohio', 'Ohio', 'Colorado'],
    ['Green', 'Red', 'Green']])
frame #注意hierarchical index，先写在前面的是第一级，也就是最左边或者说最上边

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names = ['key1', 'key2'] #index的命名顺序，从左到右
frame.columns.names = ['state', 'color'] #columns的命名顺序，从上到下
frame #下面这个diagram要注意分清column的名字和index的名字，它们叠在一起不是很直观

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

## Reordering and Sorting Levels

.swaplevel()

In [12]:
#两极反转
frame.swaplevel('key1', 'key2') #指明让哪两个反转

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

.sort_index()

In [13]:
frame.sort_index(level=1) #level=1是指对着第二个index进行排序，所以可以看到 'key2'的值是sorted的

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [14]:
frame.swaplevel(0, 1).sort_index(level=0)
#level不仅可以用str来表示，也可以用数字，调换位置之后，level=0是key2，所以除了键值不一样之外，column里面的内容结果和上面一致

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by level

.sum()

可以specify level是多少（column和row均可，只需注意axis=啥）

In [15]:
frame.sum(level='key2') #以key2为基准，sum
#key2 就俩值，1和2，所以以这两个加起来得出下面。有点类似.groupby().sum()欸

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [17]:
frame.sum(level='color', axis=1)
#以column的color为基准

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with a DataFrame's columns

.set_index()  有时候你会需要把column挪到row作为index

.reset_index()  把挪到row的index，挪回去


In [19]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
    'c': ['one', 'one', 'one', 'two', 'two',
    'two', 'two'],
    'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [20]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [21]:
frame.set_index(['c', 'd'], drop=False) #不要把挪的column删除

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [22]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
